<a href="https://colab.research.google.com/github/iamdraza/COMP8240_Major_Project/blob/main/New%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
import torch
import re

In [ ]:
!pip3 install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html


In [ ]:
!pip install sentencepiece

In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
from bs4 import BeautifulSoup
from urllib.request import urlopen
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [ ]:
model_name = 'google/pegasus-xsum'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

In [ ]:
src_text = """ Joe Biden has rejected suggestions that his Democratic party's shock loss in the Virginia governor's race was a verdict on his presidency.
Mr Biden argued that the miring of his legislative agenda in a Capitol Hill logjam did not sway the outcome. He cited "Trump voters", schools, jobs and petrol
prices as other reasons for the surprise defeat of Terry McAuliffe. Republican Glenn Youngkin, a political newcomer, came from behind to win the Virginia governor's
race on Tuesday. Meanwhile, the New Jersey governor's race went down to a nail-biting dead heat between the Democratic incumbent, Phil Murphy, and a little-known Republican
challenger, Jack Ciattarelli.
"""
batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
tgt_text

["Former US Vice-President Joe Biden has rejected suggestions that his Democratic party's shock loss in the Virginia governor's race was a verdict on his presidency."]

In [ ]:
!pip install rouge-score

In [ ]:
human_Summary =["Indian festival name called Diwali shows dazzling display of lights and colours. Millions of people celebrate this festival name Diwali of Hindu religion. But it comes amid concerns over the coronavirus pandemic and air pollution. The symbolism seems particularly meaningful at a time when coronavirus continues to disrupt peoples lives. Although cases have fallen considerably over the past month, India is still one of the world's worst hit nations, officially recording more than 35 million cases and over 450,000 deaths.",
                "The minister of Israel Karine Elharrar  has made headlines throughout the world that she couldnt attend the COP26 summit on Monday because of not accessible of Wheelchair. This behaviour looked amazed or ignored through the summit for all the disabled people and this lead to left out the conversion of climate change.",
                "There is a good chance the character of Schitts Creek have turned up on your phone even if you havent watched. The Tv show has won prestigious awards and critical acclaim, but some of its most successful marketing has come from memes.",
                "A former porter at the exclusive Abbots House apartment block in Kensington remembers meeting Dr Ruja Ignatova in 2016, as she returned from a shopping trip with her Bulgarian bodyguards. These two poor men came behind her like overloaded donkeys, struggling, and a bit out of breath - they must have had 20 bags each",
                "In pledges made at the COP26 climate summit, more than 40 countries have committed to shift away from coal. Major coal-using countries including Poland, Vietnam and Chile are among those to make the commitment.",
                "some of the worlds biggest coal dependent countries, including China and the US, did not sign up. In a separate commitment, 20 countries, including the US, pledged to end public financing for unabated fossil fuel projects abroad by the end of 2022.",
                "Joe Biden has rejected suggestions that his Democratic partys shock loss in the Virginia governor's race was a verdict on his presidency. Mr Biden argued that the miring of his legislative agenda in a Capitol Hill logjam did not sway the outcome",
                "Almost every day for 40 years, expert diver Jim Abernethy has been swimming with sharks at a patch of shallow crystal-clear ocean in the Bahamas known as Tiger Beach. At any given moment Jim can be surrounded by 30 or more tiger, reef and lemon sharks. But Jim has a unique relationship with these underwater predators who swim up to him for affectionate nose-rubs.",
                "A four year old girl missing from 18 days in a remote part of Western Australia  who has been found alive and well in a locked house after police investigation. Cleo Smith disappeared from her familys tent at a campsite near the town of Carnarvon on 16 October. Police took a 36-year-old man in custody for the investigation. Police smashed their way into a home in Carnarvon in the early hours of Wednesday, following forensic clues. One of the officers picked her up into his arms and asked her, She said, My name is Cleo. The girl has been reunited with her parents, who had made desperate pleas for Cleos return. Our family is whole again, her mother, Ellie Smith, wrote on Instagram. Police footage of Cleos rescue showed her smiling and as well as we could expect in the circumstances, said Commissioner Chris Dawson, who added she was receiving medical care. Authorities say the man in custody has no connection to the Smith family.",
                "Tigrays rebel forces currently have the upper hand in the war that erupted a year ago in northern Ethiopia. Prime Minister Abiy Ahmed, who fell out with the governing party of Tigray over his political reforms, has declared a nationwide state of emergency - it is fear and uncertainty that now rule. As rebels advance towards the capital, the government has asked residents of Addis Ababa to mobilise and protect their neighbourhoods. Fighters from Tigray, led by the Tigray People's Liberation Front (TPLF), took the cities of Dessie and Kombolcha over the weekend. They are in the Amhara region, which neighbours Tigray, and are about 400km (250 miles) from the capital. The battle for Dessie was believed to have been one of the most ferocious in the war as the city is seen as the gateway to Addis Ababa, in the south, and the border with Djibouti, in the east."]

In [ ]:
human_Summary

["Indian festival name called Diwali shows dazzling display of lights and colours. Millions of people celebrate this festival name Diwali of Hindu religion. But it comes amid concerns over the coronavirus pandemic and air pollution. The symbolism seems particularly meaningful at a time when coronavirus continues to disrupt peoples lives. Although cases have fallen considerably over the past month, India is still one of the world's worst hit nations, officially recording more than 35 million cases and over 450,000 deaths.",
 'The minister of Israel Karine Elharrar  has made headlines throughout the world that she couldnt attend the COP26 summit on Monday because of not accessible of Wheelchair. This behaviour looked amazed or ignored through the summit for all the disabled people and this lead to left out the conversion of climate change.',
 'There is a good chance the character of Schitts Creek have turned up on your phone even if you havent watched. The Tv show has won prestigious awa

In [ ]:
def preprocess_data(text):
  text_tokens = re.sub(r'\[.*?]+', '', text)
  text_tokens = (re.sub('[0-9]', '', text_tokens))
  text_tokens = re.sub('[^a-zA-Z]+', ' ', text_tokens)
  return text_tokens

In [ ]:
src = []
def scrap_webpages(link):
    'input link, it will scrap the data from the link and store in variable'
    url = link
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    text = ""
    title = ""
    for paragraph in soup.find_all('p'):
        text += paragraph.get_text()
        process_text=preprocess_data(text)
        process_text = ' '.join(process_text.split(' ')) + '.'
    src.append(preprocess_data(text))


In [ ]:
links_list = ['https://www.bbc.com/news/world-asia-india-59133207',
              'https://www.bbc.com/news/disability-59042087',
              'https://www.bbc.com/news/newsbeat-59149046',
              'https://www.bbc.com/news/stories-59062959',
              'https://www.bbc.com/news/world-europe-59075043',
              'https://www.bbc.com/news/science-environment-59159018',
              'https://www.bbc.com/news/world-us-canada-59159008',
              'https://www.bbc.com/news/av/science-environment-59157131',
              'https://www.bbc.com/news/world-australia-59143494',
              'https://www.bbc.com/news/world-africa-59151370']

In [ ]:
for links in links_list:
  #print(links)
  scrap_webpages(links)

In [ ]:
src

['India has been lit up in a dazzling display of light and colour as millions of people celebrate the Hindu festival of Diwali But it comes amid concerns over the coronavirus pandemic and air pollution A time for feasts prayers and fireworks Diwali is one of the most important festivals in India It is known as the festival of lights as people illuminate oil lamps or candles to symbolise the triumph of light over darkness and good over evil The symbolism seems particularly meaningful at a time when coronavirus continues to disrupt people s lives Although cases have fallen considerably over the past month India is still one of the world s worst hit nations officially recording more than million cases and over deaths For some Diwali marks the beginning of a new year It is also the social highpoint for Indians as people buoyed by festive cheer throw parties meet friends and family and exchange gifts The exact dates of the festival change each year and are determined by the position of the 

In [ ]:
print(len(human_Summary))

10


In [ ]:
predicted = []
def summary_gen(text):
  batch = tokenizer(text, truncation=True, padding='longest', return_tensors="pt").to(device)
  translated = model.generate(**batch)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  predicted.append(tgt_text)

In [ ]:
for i in src:
  summary_gen(i)

In [ ]:
from rouge_score import rouge_scorer
recall = []
precision = []
fmeasure = []
human_evaluation = [1, 1, 2, 2, 1, 1, 4, 3, 3, 1]
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)



for j in range(len(human_Summary)):
  scores = scorer.score(str(human_Summary[j]),str(predicted[j]))
  recall.append(scores['rouge1'].recall)
  precision.append(scores['rouge1'].precision)
  fmeasure.append(scores['rouge1'].fmeasure)

In [ ]:
source = ['bbc'] *10 

len(['xsum']*10)

10

In [ ]:
dataframe = pd.DataFrame(data= {'model': ['xsum']*len(source),
                         'source' : source,
                         'original summary': human_Summary,
                         'predicted' :clean_predicted,
                         'recall': recall,
                         'precision': precision,
                         'fmeasure': fmeasure,
                         'human_evaluation': human_evaluation
                        })

In [ ]:
dataframe

,model,source,original summary,predicted,recall,precision,fmeasure,human_evaluation
0,xsum,bbc,Indian festival name called Diwali shows dazzl...,"Images courtesy of AFP, EPA, Getty Images and ...",0.024390,0.222222,0.043956,1
1,xsum,bbc,The minister of Israel Karine Elharrar has ma...,"Disabled people are among those most ""adversel...",0.115385,0.315789,0.169014,1
2,xsum,bbc,There is a good chance the character of Schitt...,If you've been on social media in the last few...,0.214286,0.428571,0.285714,2
3,xsum,bbc,A former porter at the exclusive Abbots House ...,She's one of Europe's most notorious fraudster...,0.203704,0.207547,0.205607,2
4,xsum,bbc,"In pledges made at the COP26 climate summit, m...",As world leaders gather in Scotland for a key ...,0.142857,0.227273,0.175439,1
5,xsum,bbc,some of the worlds biggest coal dependent coun...,"The world is ""moving in the right direction"" a...",0.000000,0.000000,0.000000,1
6,xsum,bbc,Joe Biden has rejected suggestions that his De...,"""I'm not sure I would be able to have changed ...",0.534884,0.851852,0.657143,4
7,xsum,bbc,"Almost every day for 40 years, expert diver Ji...",Jim Abernethy has been swimming with sharks at...,0.439394,0.674419,0.532110,3
8,xsum,bbc,A four year old girl missing from 18 days in a...,All images are copyrighted.,0.269461,0.900000,0.414747,3
9,xsum,bbc,Tigrays rebel forces currently have the upper ...,"The Ethiopian capital, Addis Ababa, is on edge.",0.073333,0.785714,0.134146,1


In [ ]:
drive.mount('drive')
dataframe.to_csv('danish.csv')
!cp danish.csv "/content/drive/MyDrive/ADS Final/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
